# References
1. [64-bit Linux classic stack smashing tutorial using GDB-peda](https://blog.techorganic.com/2015/04/10/64-bit-linux-stack-smashing-tutorial-part-1/)
2. [64-bit Linux stack smashing tutorial with NX enabled using GDB-peda](https://blog.techorganic.com/2015/04/21/64-bit-linux-stack-smashing-tutorial-part-2/)
3. [64-bit Linux stack smashing tutorial with NX enabled and ASLR using GDB-peda](https://blog.techorganic.com/2016/03/18/64-bit-linux-stack-smashing-tutorial-part-3/)

# Stack Canaries

> This method works by placing a small integer, the value of which is randomly chosen at program start, in memory just before the stack return pointer. The stack canary value is checked to make sure it has not changed before a routine uses the return pointer on the stack.

In order to defeat this, you have to find a way to leak the canary so that you can write it back in.

# NX

[More details](https://security.stackexchange.com/questions/47807/nx-bit-does-it-protect-the-stack/47825) 

> The No-eXecute bit is a feature of the Memory Management Unit of some CPU. It will mark pages of memory as either executable or non-executable. By default, the main stack is not executable. The NX bit is set for the stack pages.

## Return-Oriented Programming (ROP)
Used to defeat NX.

> A return-oriented programming attack does not inject malicious code, but rather uses instructions that are already present, called "gadgets", by manipulating return addresses.

# Defeating NX + ASLR

### Global Offset Table (GOT) && Procedural Linkage Table (PLT)
[In depth details](https://www.technovelty.org/linux/plt-and-got-the-key-to-code-sharing-and-dynamic-libraries.html)

> PLT is used to call external procedures/functions whose address isn't known in the time of linking, and is left to be resolved by the dynamic linker at run time.

> GOT is similarly used to resolve addresses. 

> In a nutshell, the loader will initially point the GOT entry for a library function to some code that will do a slow lookup of the function address. Once it finds it, it overwrites its GOT entry with the address of the library function so it doesn’t need to do the lookup again. That means the second time a library function is called, the GOT entry will point to that function’s address. That’s what we want to leak.

![GOT/PLT diagram](https://image.slidesharecdn.com/linkerandloader-upload-160705134416/95/linker-and-loader-upload-21-638.jpg?cb=1467726311)

### Exploit

1. Use ROP to leak address of a library function in the GOT
   The gadgets will have to come from the main executable.
2. Calculate the library's base address using offsets: LIBC_BASE = ELF_FUNC_OFFSET - LIBC_FUNC_OFFSET
3. Other functions addresses can be calculated by LIBC_BASE + LIBC_FUNCi_OFFSET
4. 
   1. Find "/bash/sh" string and system function, then use ROP to call system("/bash/sh")
   2. Force executable to read from stdin, where you input "/bash/sh". Then overwrite a GOT entry to point to system. Get program to call overwritten GOT entry.

# Interesting functions for buffer overflows

1. read(0, buffer, buffer_len)

# Past CTF Challenges
1. 2017 CSAW PWN pilot : x64 classic buffer overflow
2. 2017 CSAW PWN scv : x64 buffer overflow with stack canary and ROP